# Automated Feature Engineering (paralleled version)

In [1]:
import json
import os
import warnings

import dask
import dask.bag as db
import featuretools as ft
import numpy as np
import pandas as pd
from IPython.display import display
from dask.distributed import Client, LocalCluster, Worker, get_task_stream
from feature_engine.creation import CyclicalFeatures
from feature_engine.encoding import OneHotEncoder, RareLabelEncoder
from feature_engine.imputation import AddMissingIndicator, MeanMedianImputer
from featuretools.primitives import TimeSinceFirst, TimeSinceLast
from tsfresh import extract_features
from woodwork.logical_types import Boolean, BooleanNullable, Categorical, Unknown

from utils import add_calendar_values, read_df_dtype, write_df_dtype

In [2]:
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)

In [3]:
warnings.simplefilter(action='ignore', category=FutureWarning)

## Create EntitySet

In [ ]:
"""Clients"""

from utils import read_clients


def add_clients_dataframes(es, path):
    clients_df = read_clients(path, encode_bool=False)
    clients_df = clients_df.astype({'client_id': str})
    display(clients_df.info())

    es.add_dataframe(
        clients_df,
        dataframe_name='clients',
        index='client_id',
        time_index='communication_month',
        # make_index=True,
        logical_types={
            'client_id': Unknown,
            'target': Boolean,
            'is_train': Boolean,
        }
    )
    display(es['clients'].ww.schema)
    display(es)

In [ ]:
"""Transactions"""

from utils import read_transactions


def add_transactions_dataframe(es, path):
    transactions_df = read_transactions(path, encode_bool=False, encode_category=False)
    transactions_df = transactions_df.astype({'client_id': str})
    # transactions_df['tran_date_str'] = transactions_df['tran_date'].dt.date.astype(str)  # used later for feature generation
    transactions_df = add_calendar_values(transactions_df, 'tran_date', prefix='tran_date_')
    display(transactions_df.info())

    es.add_dataframe(
        transactions_df,
        dataframe_name='transactions',
        index='transaction_id',
        time_index='tran_date',
        make_index=True,
        logical_types={
            'client_id': Unknown,
            'cat_c2': Categorical,
            'cat_c3': Categorical,
            'cat_c4': Categorical,
            # 'tran_date_str': Categorical,
            'fl_c6': Boolean,
            'fl_c7': Boolean,
            'fl_c8': Boolean,
            'fl_c9': Boolean,
            'fl_c10': Boolean,
            'fl_c11': Boolean,
            'fl_c12': Boolean,
            'fl_c13': Boolean,
            'fl_c14': Boolean,
            'fl_c15': Boolean,
        }
    )
    display(es['transactions'].ww.schema)

    es.add_relationship('clients', 'client_id', 'transactions', 'client_id')
    display(es)

In [ ]:
"""App activity"""

from utils import preprocess_app_activity_data, read_app_activity


def add_activities_dataframe(es, path):
    activities_df = read_app_activity(path, encode_bool=False, encode_category=False)
    activities_df = activities_df.astype({'client_id': str})
    activities_df = preprocess_app_activity_data(activities_df)
    # activities_df['activity_date_str'] = activities_df['activity_date'].dt.date.astype(str)  # used later for feature generation
    activities_df = add_calendar_values(activities_df, 'activity_date', prefix='activity_date_')
    display(activities_df.info())

    es.add_dataframe(
        activities_df,
        dataframe_name='activities',
        index='activity_id',
        time_index='activity_date',
        make_index=True,
        logical_types={
            'client_id': Unknown,
            'cat_c3': Categorical,
            'cat_c4': Categorical,
            'cat_c5': Categorical,
            'cat_c6': Categorical,
            'cat_c9': Categorical,
            # 'activity_date_str': Categorical,
            'cat_c8': BooleanNullable,
            'cat_c10': BooleanNullable,
        }
    )
    display(es['activities'].ww.schema)

    es.add_relationship('clients', 'client_id', 'activities', 'client_id')
    display(es)

In [ ]:
"""Communications"""

from utils import read_communications


def add_communications_dataframe(es, path):
    comms_df = read_communications(path, encode_category=False)
    comms_df = comms_df.astype({'client_id': str})
    # comms_df = preprocess_comm_data(comms)
    # comms_df = encode_comm_categories(comms)
    # comms_df['contact_date_str'] = comms_df['contact_date'].dt.date.astype(str)  # used later for feature generation
    comms_df = add_calendar_values(comms_df, 'contact_date', prefix='contact_date_')
    display(comms_df.info(show_counts=True))

    es.add_dataframe(
        comms_df,
        dataframe_name='comms',
        index='comm_id',
        time_index='contact_date',
        make_index=True,
        logical_types={
            'client_id': Unknown,
            'cat_c2': Categorical,
            'cat_c3': Categorical,
            'cat_c4': Categorical,
            'cat_c5': Categorical,
            # 'contact_date_str': Categorical,
        }
    )
    display(es['comms'].ww.schema)

    es.add_relationship('clients', 'client_id', 'comms', 'client_id')
    display(es)

In [ ]:
def entityset_from_partition(chunk_num: int, chunks_dir):
    es = ft.EntitySet(id='client_data')
    chunk_dir = os.path.join(chunks_dir, str(chunk_num))

    clients_path = os.path.join(chunk_dir, f'CLIENTS.csv')
    transactions_path = os.path.join(chunk_dir, f'TRANSACTIONS.csv')
    activities_path = os.path.join(chunk_dir, f'APP_ACTIVITY.csv')
    comms_path = os.path.join(chunk_dir, f'COMMUNICATIONS.csv')

    add_clients_dataframes(es, clients_path)
    add_transactions_dataframe(es, transactions_path)
    add_activities_dataframe(es, activities_path)
    add_communications_dataframe(es, comms_path)

    es.add_last_time_indexes()

    return {
        'es': es,
        'chunk_num': chunk_num,
    }

## Deep Feature Synthesis

In [ ]:
"""Transactions"""

def prepare_dfs_transactions(es, primitive_options):
    df_name = 'transactions'

    # Date
    for prim in ['first', 'last']:
        primitive_options[prim]['include_columns'][df_name] += ['tran_date']
    primitive_options['mean']['include_columns'][df_name] += [
        'tran_date_day_of_week_sin', 'tran_date_day_of_week_cos',
        'tran_date_day_of_month_sin', 'tran_date_day_of_month_cos',
    ]

    # Numeric
    num_cols = ['float_c16', 'float_c17', 'float_c18', 'int_c19', 'float_c20', 'float_c21']
    for prim in ['mean', 'std', 'skew', 'kurtosis', 'min', 'max', 'sum']:
        primitive_options[prim]['include_columns'][df_name] += num_cols
        # primitive_options[prim] = {'include_columns': {df_name: num_cols}}

    # Categorical
    cat_cols = list(es[df_name].ww.select(Categorical).columns)
    for prim in ['num_unique', 'mode', 'entropy']:
        # primitive_options['mode'] = {'include_columns': {df_name: cat_cols}}
        primitive_options[prim]['include_columns'][df_name] += cat_cols
    # for prim in ('mode', 'entropy'):
    #     # Delete redundant primitive
    #     primitive_options[prim]['include_columns'][df_name].remove('tran_date_str')

    # Boolean
    bool_cols = list(es[df_name].ww.select([Boolean, BooleanNullable]).columns)
    # primitive_options['percent_true'] = {'include_columns': {df_name: bool_cols}}
    primitive_options['percent_true']['include_columns'][df_name] += bool_cols


    # Interesting values
    interesting_values = {
        'cat_c2': ['4', '14', '15'],
        'cat_c3': ['209', '303', '305', '314'],
        'int_c19': [-1, 1],
        'fl_c12': [True, False],
        'fl_c13': [True, False],
        'fl_c14': [True, False],
        'tran_date_is_weekend': [True, False],
    }
    es.add_interesting_values(dataframe_name=df_name, values=interesting_values)

In [ ]:
"""Activities"""

def prepare_dfs_activities(es, primitive_options):
    df_name = 'activities'

    # Date
    for prim in ['first', 'last']:
        primitive_options[prim]['include_columns'][df_name] += ['activity_date']
    primitive_options['mean']['include_columns'][df_name] += [
        'activity_date_day_of_week_sin', 'activity_date_day_of_week_cos',
        'activity_date_day_of_month_sin', 'activity_date_day_of_month_cos',
    ]

    # Numeric
    num_cols = ['float_c11', 'float_c12', 'float_c13', 'float_c14', 'float_c15', 'float_c16', 'float_c17']
    for prim in ['mean', 'std', 'skew', 'kurtosis', 'min', 'max', 'sum']:
        primitive_options[prim]['include_columns'][df_name] += num_cols

    # Categorical
    cat_cols = list(es[df_name].ww.select(Categorical).columns)
    for prim in ['num_unique', 'mode', 'entropy']:
        primitive_options[prim]['include_columns'][df_name] += cat_cols
    # for prim in ('mode', 'entropy'):
    #     # Delete redundant primitive
    #     primitive_options[prim]['include_columns'][df_name].remove('activity_date_str')

    # Boolean
    bool_cols = list(es[df_name].ww.select([Boolean, BooleanNullable]).columns)
    primitive_options['percent_true']['include_columns'][df_name] += bool_cols


    # Interesting values
    interesting_values = {
        'cat_c4': ['1', '2'],
        'cat_c6': ['1', '2', '3'],
        'cat_c9': ['1', '2'],
        'cat_c8': [True, False],
        'cat_c10': [True, False],
        'activity_date_is_weekend': [True, False],
    }
    es.add_interesting_values(dataframe_name=df_name, values=interesting_values)

In [ ]:
"""Communications"""

def prepare_dfs_communications(es, primitive_options):
    df_name = 'comms'

    # Date
    for prim in ['first', 'last']:
        primitive_options[prim]['include_columns'][df_name] += ['contact_date']
    primitive_options['mean']['include_columns'][df_name] += [
        'contact_date_day_of_week_sin', 'contact_date_day_of_week_cos',
        'contact_date_day_of_month_sin', 'contact_date_day_of_month_cos',
    ]

    # Categorical
    cat_cols = list(es[df_name].ww.select(Categorical).columns)
    for prim in ['num_unique', 'mode', 'entropy']:
        primitive_options[prim]['include_columns'][df_name] += cat_cols
    # for prim in ('mode', 'entropy'):
    #     # Delete redundant primitive
    #     primitive_options[prim]['include_columns'][df_name].remove('contact_date_str')


    # Boolean
    bool_cols = list(es[df_name].ww.select([Boolean, BooleanNullable]).columns)
    primitive_options['percent_true']['include_columns'][df_name] += bool_cols

    # Interesting values
    interesting_values = {
        'cat_c2': ['S3564', 'S3565', 'S3677', 'S3769'],
        'cat_c3': ['3', '4', '7'],
        'cat_c4': ['1', '2'],
        'cat_c5': ['4', '7'],
        'contact_date_is_weekend': [True, False],
    }
    es.add_interesting_values(dataframe_name=df_name, values=interesting_values)

In [ ]:
AGG_PRIMITIVES = [
    'count',
    'first', 'last', TimeSinceFirst(unit='days'), TimeSinceLast(unit='days'),  # for date
    'mean', 'std', 'skew', 'kurtosis', 'min', 'max', 'sum',   # for numeric
    'num_unique', 'mode', 'entropy',  # for categorical
    'percent_true',  # for boolean
]

def prepare_dfs(es):
    primitive_options = {}
    for agg in AGG_PRIMITIVES:
        primitive_options[agg]= {'include_columns': {}}
        for df_name in ('transactions', 'activities', 'comms'):
             if agg not in ('count',) and isinstance(agg, str):
                 primitive_options[agg]['include_columns'][df_name] = []

    prepare_dfs_transactions(es, primitive_options)
    prepare_dfs_activities(es, primitive_options)
    prepare_dfs_communications(es, primitive_options)

    return primitive_options

In [ ]:
"""Cut-off time dataframe"""

def get_cutoff_time(es):
    cutoff_df = es.dataframe_dict['clients'][['client_id', 'communication_month']].copy()
    cutoff_df.rename(columns={'communication_month': 'time'}, inplace=True)
    return cutoff_df

In [ ]:
def run_dfs(es, primitive_options, cutoff_df):
    feature_matrix, feature_defs = ft.dfs(
        entityset=es,
        target_dataframe_name='clients',
        cutoff_time=cutoff_df,
        agg_primitives=AGG_PRIMITIVES,
        trans_primitives=['day', 'month', 'weekday', 'is_weekend'],
        where_primitives=['mean', 'sum', 'count'],
        primitive_options=primitive_options,
        max_depth=1,
        verbose=True,
        features_only=False,
        n_jobs=1,
        chunk_size=es.dataframe_dict['clients'].shape[0],
        return_types='all',
    )
    # feature_matrix_enc, features_enc = ft.encode_features(
    #     feature_matrix,
    #     feature_defs,
    #     top_n={
    #         # Transactions
    #         'MODE(transactions.cat_c2)': 10,
    #         'MODE(transactions.cat_c3)': 10,
    #         'MODE(transactions.cat_c4)': 5,
    #
    #         # Activities
    #         'MODE(activities.cat_c3)': 3,
    #         'MODE(activities.cat_c4)': 2,
    #         'MODE(activities.cat_c5)': 1,
    #         'MODE(activities.cat_c6)': 7,
    #         'MODE(activities.cat_c9)': 2,
    #
    #         # Communications
    #         'MODE(comms.cat_c2)': 10,
    #         'MODE(comms.cat_c3)': 3,
    #         'MODE(comms.cat_c4)': 3,
    #         'MODE(comms.cat_c5)': 3,
    #     },
    #     to_encode=[
    #         'MODE(transactions.cat_c2)', 'MODE(transactions.cat_c3)', 'MODE(transactions.cat_c4)',  # transactions
    #         'MODE(activities.cat_c3)', 'MODE(activities.cat_c4)', 'MODE(activities.cat_c5)', 'MODE(activities.cat_c6)', 'MODE(activities.cat_c9)',  # activities
    #         'MODE(comms.cat_c2)', 'MODE(comms.cat_c3)', 'MODE(comms.cat_c4)', 'MODE(comms.cat_c5)',  # communications
    #     ],
    # )

    return feature_matrix, feature_defs

## Ratio of periods features

In [ ]:
def compute_ratios_for_periods(es, fm, cutoff_df, primitive_options):
    """Ratios between last 45/90 days periods"""
    fm_45d, _ = ft.dfs(
        entityset=es,
        target_dataframe_name='clients',
        cutoff_time=cutoff_df,
        training_window='45 days',
        agg_primitives=['count', 'sum', 'mean', 'std', 'percent_true'],
        trans_primitives=[],
        where_primitives=['count'],
        primitive_options=primitive_options,
        max_depth=1,
        verbose=True,
        features_only=False,
        n_jobs=1,
        chunk_size=es.dataframe_dict['clients'].shape[0],
        return_types='all',
    )
    fm_90d, _ = ft.dfs(
        entityset=es,
        target_dataframe_name='clients',
        cutoff_time=cutoff_df,
        training_window='90 days',
        agg_primitives=['count', 'sum', 'mean', 'std', 'percent_true'],
        trans_primitives=[],
        where_primitives=['count'],
        primitive_options=primitive_options,
        max_depth=1,
        verbose=True,
        features_only=False,
        n_jobs=1,
        chunk_size=es.dataframe_dict['clients'].shape[0],
        return_types='all',
    )

    # Rename to make columns unique
    fm_45d = fm_45d.add_prefix('D45_')
    fm_90d = fm_90d.add_prefix('D90_')

    trend_features = pd.DataFrame(index=fm_45d.index)
    for col_45, col_90 in zip(fm_45d.columns, fm_90d.columns):
        col = col_45.replace('D45_', '')
        if col in ['client_id', 'target', 'is_train']:
            continue

        if pd.api.types.is_numeric_dtype(fm_45d[col_45]):
            trend_features[f"TREND_45_90_{col}"] = fm_45d[col_45] / fm_90d[col_90].replace(0, np.nan)

    trend_features.fillna(0, inplace=True)

    display(trend_features.head())
    display(trend_features.info(verbose=True, show_counts=True))

    return fm.merge(trend_features, on='client_id', how='left')

## Other features

In [ ]:
def add_extra_unique_features(es, fm):
    transactions_df = es.dataframe_dict['transactions']
    activities_df = es.dataframe_dict['activities']
    comms_df = es.dataframe_dict['comms']

    transactions_df['tran_date_str'] = transactions_df['tran_date'].dt.date.astype('category')
    tx_dates_unique = transactions_df.groupby('client_id').agg(tran_date_str_nunique=('tran_date_str', 'nunique')).reset_index(names='client_id')

    activities_df['activity_date_str'] = activities_df['activity_date'].dt.date.astype('category')
    act_dates_unique = activities_df.groupby('client_id').agg(activity_date_str_nunique=('activity_date_str', 'nunique')).reset_index(names='client_id')
    act_devices_unique = activities_df.groupby('client_id').agg(device_id_nunique=('device_id', 'nunique')).reset_index(names='client_id')

    comms_df['contact_date_str'] = comms_df['contact_date'].dt.date.astype('category')
    comms_dates_unique = comms_df.groupby('client_id').agg(contact_date_str_nunique=('contact_date_str', 'nunique')).reset_index(names='client_id')

    fm = fm.merge(comms_dates_unique, on='client_id', how='left')
    fm = fm.merge(act_dates_unique, on='client_id', how='left')
    fm = fm.merge(act_devices_unique, on='client_id', how='left')
    fm = fm.merge(tx_dates_unique, on='client_id', how='left')

    return fm

In [ ]:
def add_cross_table_interaction_features(es, fm):
    fm['INTERACTION_activity_per_trans'] = fm['COUNT(activities)'] / fm['COUNT(transactions)'].replace(0, 1)
    fm['INTERACTION_contact_per_trans'] = fm['COUNT(comms)'] / fm['COUNT(transactions)'].replace(0, 1)
    fm['INTERACTION_contact_per_activity'] = fm['COUNT(comms)'] / fm['COUNT(activities)'].replace(0, 1)
    return fm

In [ ]:
def add_financial_balance_features(es, fm):
    pos_sum = fm['SUM(transactions.float_c18 WHERE int_c19 = 1)']
    neg_sum = fm['SUM(transactions.float_c18 WHERE int_c19 = -1)']
    fm['BALANCE_net_flow'] = pos_sum - neg_sum
    fm['BALANCE_savings_potential'] = pos_sum / neg_sum.replace(0, 1)
    return fm

## Add more time-series features

In [ ]:
def prep_time_gaps(df, date_col, prefix):
    df = df.sort_values(by=['client_id', date_col])

    gap_col_name = f'{prefix}_gap_days'
    df[gap_col_name] = df.groupby('client_id')[date_col].diff().dt.total_seconds() / (3600 * 24)
    df[gap_col_name] = df[gap_col_name].fillna(0)

    return df[['client_id', date_col, gap_col_name]]


def add_time_series_features(es, fm):
    transactions_df = es.dataframe_dict['transactions']
    activities_df = es.dataframe_dict['activities']
    comms_df = es.dataframe_dict['comms']

    df_transactions_ts = prep_time_gaps(transactions_df, 'tran_date', 'tran_date')
    df_transactions_ts['float_c18_diff'] = transactions_df.groupby('client_id')['float_c18'].diff().fillna(0)  # difference between transactions amount
    df_activities_ts = prep_time_gaps(activities_df, 'activity_date', 'activity_date')
    df_comms_ts  = prep_time_gaps(comms_df, 'contact_date', 'contact_date')

    display(df_transactions_ts.head(10))

    datasets = {
        'transactions': (df_transactions_ts, 'tran_date', ['tran_date_gap_days', 'float_c18_diff']),
        'activities': (df_activities_ts, 'activity_date', ['activity_date_gap_days']),
        'communications': (df_comms_ts, 'contact_date', ['contact_date_gap_days'])
    }
    all_features = []
    for name, (df, date_col, val_cols) in datasets.items():
        print(f'Processing {name}...')

        for val_col in val_cols:
            fc_parameters = {
                'median': None,
                'mean': None,
                'maximum': None,
                'minimum': None,
                'standard_deviation': None,
                'skewness': None,
                'kurtosis': None,
                'quantile': [
                    {'q': 0.1},
                    {'q': 0.25},
                    {'q': 0.75},
                    {'q': 0.9}
                ],
                'linear_trend': [{'attr': 'slope'}],
                'approximate_entropy': [{'m': 2, 'r': 0.25}],
                'ratio_beyond_r_sigma': [{'r': 2}],
                'autocorrelation': [{'lag': 1}],
                'fft_aggregated': [{'aggtype': 'centroid'}, {'aggtype': 'variance'}],
                'longest_strike_above_mean': None,
                'longest_strike_below_mean': None,
                'count_above_mean': None,
            }
            features = extract_features(
                timeseries_container=df,
                column_id='client_id',
                column_sort=date_col,
                column_value=val_col,
                default_fc_parameters=fc_parameters,
                n_jobs=1,
                # chunksize=es.dataframe_dict['clients'].shape[0] // 3 + 1,
                chunksize=es.dataframe_dict['clients'].shape[0],
            )
            features = features.reset_index(names='client_id')
            all_features.append(features)

    for features in all_features:
        fm = fm.merge(features, on='client_id', how='left')

    return fm

## Main method

In [ ]:
CHUNKS_DIR = '../data/chunks_54/'
FM_OUTPUT_DIR = '../data/fm/'

In [ ]:
def feature_matrix_from_entityset(es_dict, save=True):
    es = es_dict['es']
    chunk_num = es_dict['chunk_num']

    primitive_options = prepare_dfs(es)
    cutoff_df = get_cutoff_time(es)

    fm, fd = run_dfs(es, primitive_options, cutoff_df)
    display(fd)
    display(fm.head())
    display(fm.info(verbose=True, show_counts=True))
    display(fm.describe())

    fm = compute_ratios_for_periods(es, fm, cutoff_df, primitive_options)
    fm = add_extra_unique_features(es, fm)
    fm = add_cross_table_interaction_features(es, fm)
    fm = add_financial_balance_features(es, fm)
    fm = add_time_series_features(es, fm)

    display(fm.info(verbose=True, show_counts=True))

    if save:
        # chunk_dir = os.path.join(FM_OUTPUT_DIR, str(es_dict['chunk_num']))
        # os.makedirs(chunk_dir, exist_ok=True)
        fm_output_path = os.path.join(FM_OUTPUT_DIR, f'feature_matrix_{chunk_num}.csv')
        fm.to_csv(fm_output_path, index=False)

        dtypes_output_path = os.path.join(FM_OUTPUT_DIR, f'fm_dtypes_{chunk_num}.json')
        write_df_dtype(fm, dtypes_output_path)

In [ ]:
"""For testing purposes only"""

# es_dict = entityset_from_partition(1, CHUNKS_DIR)
# fm = feature_matrix_from_entityset(es_dict, save=False)

## Compute with Dask

In [ ]:
dask.config.set({
    'distributed.worker.memory.target': 0.70,
    'distributed.worker.memory.spill': 0.85,
    'distributed.worker.memory.pause': 0.90,
    'distributed.worker.memory.terminate': False,
})
client = Client(
    n_workers=6,
    threads_per_worker=1,
    memory_limit='16GB'
)
print(f'Dask Dashboard available at: {client.dashboard_link}')

In [ ]:
CHUNKS = 54

chunks_nums = list(range(1, CHUNKS + 1))
b = dask.bag.from_sequence(chunks_nums, npartitions=len(chunks_nums))  # specify npartitions implicitly to avoid auto-grouping
b = b.map(entityset_from_partition, chunks_dir=CHUNKS_DIR)
b = b.map(feature_matrix_from_entityset)
display(b)

In [ ]:
with get_task_stream() as ts:
    result = b.compute()

history = pd.DataFrame(ts.data)
display(history)

In [ ]:
client.close()

## Final Feature Matrix

In [4]:
RAW_FM_OUT_PATH = '../data/features/features_auto_v5_raw.csv'
RAW_FM_DTYPE_OUT_PATH = '../data/features/features_auto_v5_raw_dtype.json'

In [ ]:
fms = []
for i in chunks_nums:
    path = os.path.join(FM_OUTPUT_DIR, f'feature_matrix_{i}.csv')
    dtype_path = os.path.join(FM_OUTPUT_DIR, f'fm_dtypes_{i}.json')
    fm_dtype, date_cols = read_df_dtype(dtype_path)
    fms.append(pd.read_csv(path, dtype=fm_dtype, parse_dates=date_cols))

feature_matrix = pd.concat(fms, axis=0)
feature_matrix.to_csv(RAW_FM_OUT_PATH, index=False)
write_df_dtype(feature_matrix, RAW_FM_DTYPE_OUT_PATH)

## Postprocessing

In [5]:
MIN_DATE_STR = '2024-12-01 00:00:00'
MAX_DATE_STR = '2025-09-01 00:00:00'

In [19]:
def handle_missing_values(df):
    # Drop missing
    df = df.dropna(subset=[
        # Transactions
        'tran_date_str_nunique', 'STD(transactions.float_c18)',

        # Activities
        'activity_date_str_nunique', 'STD(activities.float_c11)',

        # Communications
        'contact_date_str_nunique',
    ])

    # Fill missing
    cols_to_fill_with_mean = [
        c for c in df.columns
        if (' WHERE ' in c and c.startswith('MEAN(')
            or c.startswith('SKEW(')
            or c.endswith('__skewness')
            or c.endswith('__kurtosis')
            or '__autocorrelation' in c
            or '__fft_aggregated__aggtype' in c)
    ]
    for col in cols_to_fill_with_mean:
        if df[col].isnull().mean() > 0.05:
            ami = AddMissingIndicator(variables=[col])
            df = ami.fit_transform(df)
        mmi = MeanMedianImputer(imputation_method='mean', variables=[col])
        df = mmi.fit_transform(df)

    return df

def convert_dates(df):
    """Convert dates to days from initial point"""
    for date_col in df.select_dtypes(include=['datetime64[ns]']).columns:
        df[date_col + '_days'] = (df[date_col] - pd.to_datetime(MIN_DATE_STR)).dt.days
        df = df.drop(columns=[date_col])
    return df

def encode_cyclical_features(df):
    """Encode cyclical features (e.g. day of week, month)"""
    time_cols = ['DAY(communication_month)', 'MONTH(communication_month)', 'WEEKDAY(communication_month)']
    df = df.astype({col: 'int8' for col in time_cols})
    cyclical = CyclicalFeatures(variables=time_cols, drop_original=True)
    df = cyclical.fit_transform(df)
    return df

def encode_categorical_features(df):
    top_n_categ = {
        # Transactions
        'MODE(transactions.cat_c2)': 4,
        'MODE(transactions.cat_c3)': 6,
        'MODE(transactions.cat_c4)': 1,

        # Activities
        'MODE(activities.cat_c3)': 1,
        'MODE(activities.cat_c4)': 1,
        'MODE(activities.cat_c5)': 1,
        'MODE(activities.cat_c6)': 5,
        'MODE(activities.cat_c9)': 2,

        # Communications
        'MODE(comms.cat_c2)': 10,
        'MODE(comms.cat_c3)': 1,
        'MODE(comms.cat_c4)': 2,
        'MODE(comms.cat_c5)': 1,
    }
    for col, top_n in top_n_categ.items():
        rle = RareLabelEncoder(tol=0, n_categories=1, max_n_categories=top_n, variables=[col])
        df = rle.fit_transform(df)

    ohe = OneHotEncoder(variables=list(top_n_categ.keys()))
    df = ohe.fit_transform(df)

    return df

def convert_boolean_values(df):
    """Convert boolean values to 0 and 1"""
    df = df.astype({col: 'int8' for col in df.select_dtypes(include=['bool']).columns})
    df = df.astype({col: 'Int8' for col in df.select_dtypes(include=['boolean']).columns})
    return df

In [20]:
df_dtype, df_date_cols = read_df_dtype(RAW_FM_DTYPE_OUT_PATH)
df = pd.read_csv(RAW_FM_OUT_PATH, dtype=df_dtype, parse_dates=df_date_cols)
# df = feature_matrix

# df = df.reset_index()
# df = df.drop(columns=['client_id'])

df = handle_missing_values(df)
df = convert_dates(df)
df = encode_cyclical_features(df)
df = encode_categorical_features(df)
df = convert_boolean_values(df)

df.to_csv('../data/features/features_auto_v5.csv', index=False)
display(df.info(verbose=True, show_counts=True))
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 34964 entries, 0 to 36336
Data columns (total 983 columns):
 #    Column                                                                                  Non-Null Count  Dtype  
---   ------                                                                                  --------------  -----  
 0    client_id                                                                               34964 non-null  string 
 1    target                                                                                  34964 non-null  int8   
 2    is_train                                                                                34964 non-null  int8   
 3    COUNT(transactions)                                                                     34964 non-null  Int64  
 4    ENTROPY(transactions.cat_c2)                                                            34964 non-null  float64
 5    ENTROPY(transactions.cat_c3)                                   

None

target  is_train  COUNT(transactions)  ENTROPY(transactions.cat_c2)  \
count 34964.000 34964.000            34964.000                     34964.000   
mean      0.245     0.793              447.722                         1.574   
std       0.430     0.405              474.128                         0.403   
min       0.000     0.000                2.000                         0.000   
25%       0.000     1.000              122.000                         1.329   
50%       0.000     1.000              309.000                         1.621   
75%       0.000     1.000              616.000                         1.855   
max       1.000     1.000             8721.000                         2.721   

       ENTROPY(transactions.cat_c3)  ENTROPY(transactions.cat_c4)  \
count                     34964.000                     34964.000   
mean                          1.643                         0.106   
std                           0.453                         0.191   
min                           0.000                         0.000   
25%                           1.359                         0.000   
50%                           1.674                         0.016   
75%                           1.944                         0.127   
max                           3.290                         2.159   

       KURTOSIS(transactions.float_c16)  KURTOSIS(transactions.float_c17)  \
count                         34964.000                         34964.000   
mean                             78.719                           132.415   
std                             146.909                           225.167   
min                              -2.000                            -2.000   
25%                               0.000                             0.000   
50%                              27.624                            34.186   
75%                              93.020                           176.602   
max                            3572.596                          2741.303   

       KURTOSIS(transactions.float_c18)  KURTOSIS(transactions.float_c20)  \
count                         34964.000                         34964.000   
mean                             42.100                            16.053   
std                              57.951                           112.541   
min                              -2.000                            -2.000   
25%                              11.096                             0.000   
50%                              25.523                             0.000   
75%                              51.168                             0.000   
max                            1350.809                          3767.000   

       KURTOSIS(transactions.float_c21)  KURTOSIS(transactions.int_c19)  \
count                         34964.000                       34964.000   
mean                             43.426                          -0.719   
std                              62.417                           2.654   
min                              -2.000                          -2.000   
25%                              11.074                          -1.807   
50%                              25.791                          -1.328   
75%                              52.145                          -0.380   
max                            1588.049                         259.670   

       MAX(transactions.float_c16)  MAX(transactions.float_c17)  \
count                    34964.000                    34964.000   
mean                         1.620                        4.577   
std                          4.868                       19.215   
min                          0.000                        0.000   
25%                          0.000                        0.000   
50%                          0.470                        1.000   
75%                          1.660                        4.010   
max                        413.130                     2294.720   